## Part 2 Datasets/DataFrames: Spark ML and Pipelines
Convert the review texts to a classic vector space representation with TFIDF-weighted features based on the Spark DataFrame/Dataset API by building a transformation pipeline. The primary goal of this part is the preparation of the pipeline for Part 3 (see below). Note: although parts of this pipeline will be very similar to Assignment 1 or Part 1 above, do not expect to obtain identical results or have access to all intermediate outputs to compare the individual steps.

Use built-in functions for tokenization to unigrams at whitespaces, tabs, digits, and the delimiter characters ()[]{}.!?,;:+=-_"'`~#@&*%€$§\/, casefolding, stopword removal, TF-IDF calculation, and chi square selection ) (using 2000 top terms overall). Write the terms selected this way to a file output_ds.txt and compare them with the terms selected in Assignment 1. Describe your observations briefly in the submission report (see Part 3).

[Provided link for ML pipeline](https://spark.apache.org/docs/latest/ml-pipeline.html)  
[Provided link for feature extraction](https://spark.apache.org/docs/latest/ml-features.html)

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import StringIndexer

In [2]:
# Initialize Spark context and session
conf = SparkConf().setAppName("Part2")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
24/05/14 12:11:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/14 12:11:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/14 12:11:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/14 12:11:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/14 12:11:27 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading lib

In [3]:
spark

### Importing data
Stopwords file as well as test data

In [4]:
# Define the stopwords file and the counters file
stopwords_file = "stopwords.txt"

# Load stopwords into a set
with open(stopwords_file, "r") as f:
    stopwords = set(f.read().strip().split())
    
# Load and preprocess the Amazon reviews dataset
input_file = "hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json"
reviews_df = spark.read.json(input_file)

In [5]:
reviews_df = reviews_df.select("category", "reviewText")

# Show the DataFrame with selected columns
reviews_df.show()

+--------------------+--------------------+
|            category|          reviewText|
+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...|
|Patio_Lawn_and_Garde|This is a very ni...|
|Patio_Lawn_and_Garde|The metal base wi...|
|Patio_Lawn_and_Garde|For the most part...|
|Patio_Lawn_and_Garde|This hose is supp...|
|Patio_Lawn_and_Garde|This tool works v...|
|Patio_Lawn_and_Garde|This product is a...|
|Patio_Lawn_and_Garde|I was excited to ...|
|Patio_Lawn_and_Garde|I purchased the L...|
|Patio_Lawn_and_Garde|Never used a manu...|
|Patio_Lawn_and_Garde|Good price. Good ...|
|Patio_Lawn_and_Garde|I have owned the ...|
|Patio_Lawn_and_Garde|I had "won" a sim...|
|Patio_Lawn_and_Garde|The birds ate all...|
|Patio_Lawn_and_Garde|Bought last summe...|
|Patio_Lawn_and_Garde|I knew I had a mo...|
|Patio_Lawn_and_Garde|I was a little wo...|
|Patio_Lawn_and_Garde|I have used this ...|
|Patio_Lawn_and_Garde|I actually do not...|
|Patio_Lawn_and_Garde|Just what 

In [6]:
# Show first two objects of reviews_rdd
reviews_df.take(2)

[Row(category='Patio_Lawn_and_Garde', reviewText="This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!"),
 Row(category='Patio_Lawn_and_Garde', reviewText='This is a very nice spreader.  It feels very solid and the pneumatic tires give it great maneuverability and handling over bumps.  The control arm is solid metal, not a cable, which gives you precise control and will last a long time.  The settings take some experimentation with your various products to get it right, but that is true of any spreader.  It has good distribution... probably flings material a little 

In [7]:
pattern = r'\s+|\d+|[(){}\[\].!?,;:+=_"\'`~#@&*%€$§\\/\-]'
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern=pattern)

In [8]:
reviews_df = regexTokenizer.transform(reviews_df)

In [9]:
reviews_df.columns

['category', 'reviewText', 'words']

In [10]:
reviews_df.take(1)

[Row(category='Patio_Lawn_and_Garde', reviewText="This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!", words=['this', 'was', 'a', 'gift', 'for', 'my', 'other', 'husband', 'he', 's', 'making', 'us', 'things', 'from', 'it', 'all', 'the', 'time', 'and', 'we', 'love', 'the', 'food', 'directions', 'are', 'simple', 'easy', 'to', 'read', 'and', 'interpret', 'and', 'fun', 'to', 'make', 'we', 'all', 'love', 'different', 'kinds', 'of', 'cuisine', 'and', 'raichlen', 'provides', 'recipes', 'from', 'everywhere', 'along', 'the', 'barbecue', 'trail', 'as', 'he', 'calls', 'it', '

In [11]:
type(stopwords)

set

In [13]:
remover = StopWordsRemover(stopWords = list(stopwords), inputCol="words", outputCol="filtered_words")
reviews_df = remover.transform(reviews_df)

Add HashingTF HashingTermFrequency

In [14]:
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=2000)
reviews_df = hashingTF.transform(reviews_df)

In [15]:
reviews_df.take(1)

[Row(category='Patio_Lawn_and_Garde', reviewText="This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!", words=['this', 'was', 'a', 'gift', 'for', 'my', 'other', 'husband', 'he', 's', 'making', 'us', 'things', 'from', 'it', 'all', 'the', 'time', 'and', 'we', 'love', 'the', 'food', 'directions', 'are', 'simple', 'easy', 'to', 'read', 'and', 'interpret', 'and', 'fun', 'to', 'make', 'we', 'all', 'love', 'different', 'kinds', 'of', 'cuisine', 'and', 'raichlen', 'provides', 'recipes', 'from', 'everywhere', 'along', 'the', 'barbecue', 'trail', 'as', 'he', 'calls', 'it', '

In [16]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(reviews_df)
reviews_df = idfModel.transform(reviews_df)

In [17]:
reviews_df.take(1)

[Row(category='Patio_Lawn_and_Garde', reviewText="This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!", words=['this', 'was', 'a', 'gift', 'for', 'my', 'other', 'husband', 'he', 's', 'making', 'us', 'things', 'from', 'it', 'all', 'the', 'time', 'and', 'we', 'love', 'the', 'food', 'directions', 'are', 'simple', 'easy', 'to', 'read', 'and', 'interpret', 'and', 'fun', 'to', 'make', 'we', 'all', 'love', 'different', 'kinds', 'of', 'cuisine', 'and', 'raichlen', 'provides', 'recipes', 'from', 'everywhere', 'along', 'the', 'barbecue', 'trail', 'as', 'he', 'calls', 'it', '

To implement the Chi Square selector, we first need to convert the categories into numerics

In [18]:
# Apply StringIndexer to convert categorical column to numerical
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
reviews_df = indexer.fit(reviews_df).transform(reviews_df)

In [19]:
reviews_df.select('categoryIndex').show()

+-------------+
|categoryIndex|
+-------------+
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
|         18.0|
+-------------+
only showing top 20 rows



## TODO
Since ChiSquare Selector does not calculate the Chi square value by default but right now only selects the top n values, we need to figure out if there is a function for it or if we have to write it manually as done in Part 1.

In [20]:
selector = ChiSqSelector(numTopFeatures=75, featuresCol="features",outputCol="selectedFeatures", labelCol="categoryIndex")
result = selector.fit(reviews_df)

In [ ]:
pipeline = Pipeline(stages=[regexTokenizer, ])